In [ ]:
!pip install chromadb sentence-transformers openai tqdm numpy


In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from openai import OpenAI


In [ ]:
RAW_FILES = [
    "/content/CS_normalized.json",
    "/content/SWE_normalized.json",
    "/content/STUDENT_RULES.json",
    "/content/AI_normalized.json",
    "/content/DS_normalized.json",
    "/content/CNE_normalized.json",
    "/content/CY_normalized.json",
]



In [ ]:
def extract_chunks_from_file(path):
    import json

    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    chunks = []

    # ================
    # 1) Handle RULE CHUNKS (arrays of objects with "text")
    # ================
    if isinstance(data, list):
        for item in data:
            if isinstance(item, dict):
                if "text" in item and isinstance(item["text"], str):
                    chunks.append(item)
        # continue scanning deeply
        # (Student rules may nest further)

    # ================
    # 2) Handle PROGRAM FILES (levels → courses)
    # ================
    if isinstance(data, dict):
        program = data.get("program")
        levels = data.get("levels", [])

        # program course structure
        if isinstance(levels, list):
            for lvl in levels:
                level_id = lvl.get("level_id") or lvl.get("level")
                courses = lvl.get("courses", [])
                for c in courses:
                    if isinstance(c, dict):
                        text = f"{c.get('code','')} - {c.get('name','')} | Credits: {c.get('credits',0)}"
                        chunks.append({
                            "text": text,
                            "metadata": {
                                "program": program,
                                "level": level_id,
                                "dept": c.get("dept",""),
                                "code": c.get("code",""),
                                "credits": c.get("credits",0),
                                "prerequisites": c.get("prerequisites", [])
                            }
                        })

    # ================
    # 3) Handle AI ELECTIVE COURSES (electiveList → array)
    # ================
    if isinstance(data, dict) and "electiveList" in data:
        for item in data["electiveList"]:
            text = f"{item.get('course_code','')} - {item.get('course_name','')}"
            chunks.append({
                "text": text,
                "metadata": {
                    "category": item.get("category",""),
                    "credits": item.get("credits", 0),
                    "prerequisites": item.get("prerequisites", [])
                }
            })

    # ================
    # 4) Deep scan ANY dict/list for “text”, “description”, “name”
    # (generic fallback)
    # ================
    def deep_scan(obj):
        if isinstance(obj, dict):
            # Extract if any textual field exists
            for key in ["text", "description", "rule", "name", "title"]:
                if key in obj and isinstance(obj[key], str):
                    chunks.append({
                        "text": obj[key],
                        "metadata": {}
                    })

            for v in obj.values():
                deep_scan(v)

        elif isinstance(obj, list):
            for item in obj:
                deep_scan(item)

    deep_scan(data)

    return chunks

all_chunks = []
for file in RAW_FILES:
    if not os.path.exists(file):
        print(f"❌ File not found: {file}")
        continue

    extracted = extract_chunks_from_file(file)
    print(f"📥 {file}: {len(extracted)} chunks loaded")
    all_chunks.extend(extracted)

print("🔥 TOTAL CHUNKS LOADED:", len(all_chunks))


📥 /content/CS_normalized.json: 94 chunks loaded
📥 /content/SWE_normalized.json: 94 chunks loaded
📥 /content/STUDENT_RULES.json: 158 chunks loaded
📥 /content/AI_normalized.json: 98 chunks loaded
📥 /content/DS_normalized.json: 94 chunks loaded
📥 /content/CNE_normalized.json: 88 chunks loaded
📥 /content/CY_normalized.json: 92 chunks loaded
📥 /content/AI_Elective_Courses.json: 0 chunks loaded
🔥 TOTAL CHUNKS LOADED: 718


In [ ]:
model = SentenceTransformer("intfloat/multilingual-e5-large")


In [ ]:
from chromadb import PersistentClient

chroma = PersistentClient(path="/content/chroma_store")
collection = chroma.get_or_create_collection("rag_collection")



In [ ]:
texts = [c["text"] for c in all_chunks]
ids = [c["id"] if "id" in c else f"chunk_{i}" for i in range(len(all_chunks))]

def sanitize_metadata(md):
    clean = {}
    for k, v in md.items():
        if isinstance(v, list):
            clean[k] = ", ".join(str(x) for x in v)
        elif isinstance(v, dict):
            clean[k] = json.dumps(v, ensure_ascii=False)
        elif v is None:
            clean[k] = ""
        else:
            clean[k] = v

    # ChromaDB requires metadata to NOT be empty
    if len(clean) == 0:
        clean["source"] = "chunk"

    return clean


metadatas = [sanitize_metadata(c.get("metadata", {})) for c in all_chunks]

print("Embedding", len(texts), "chunks…")
embeddings = model.encode(texts, show_progress_bar=True)

collection.add(
    embeddings=embeddings.tolist(),
    documents=texts,
    metadatas=metadatas,
    ids=ids
)

print("🔥 ChromaDB Collection Ready!")


Embedding 718 chunks…


Batches:   0%|          | 0/23 [00:00<?, ?it/s]

🔥 ChromaDB Collection Ready!


In [ ]:
def retrieve(query, k=5):
    q_emb = model.encode([query])[0].tolist()
    results = collection.query(
        query_embeddings=[q_emb],
        n_results=k
    )
    return results


In [ ]:
def rule_score(chunk_text, expected_keywords):
    score = 0
    for kw in expected_keywords:
        if kw in chunk_text:
            score += 1
    return score / len(expected_keywords)


In [ ]:
def llm_similarity_score(query, retrieved_text, ground_truth):
    prompt = f"""
You evaluate retrieval correctness.

Query: {query}

Retrieved text:
{retrieved_text}

Ground truth:
{ground_truth}

Give a score from 0 to 100.
Return ONLY the number.
"""

    resp = client.responses.create(
        model="gpt-4.1-mini",
        input=prompt
    )

    output = resp.output_text.strip()

    try:
        return float(output)
    except:
        return 0.0



In [ ]:
def hybrid_score(query, retrieved_text, expected_keywords, ground_truth):
    rule = rule_score(retrieved_text, expected_keywords)
    llm_s = llm_similarity_score(query, retrieved_text, ground_truth) / 100
    return (rule + llm_s) / 2


In [ ]:
#example to expand
EVAL_SET = [
    {
        "query": "ما هي شروط التخرج؟",
        "keywords": ["المعدل", "التخرج"],
        "ground_truth": "يجب أن يكون المعدل التراكمي لا يقل عن المقبول …"
    },
    {
        "query": "متى يحرم الطالب من الاختبار النهائي؟",
        "keywords": ["75", "حرمان"],
        "ground_truth": "يُحرم الطالب إذا تجاوز غيابه 25٪ …"
    }
]


In [ ]:
from sentence_transformers import util
import torch


In [ ]:
def embed(text):
    return model.encode(text, convert_to_tensor=True)


In [ ]:
def semantic_score(query, retrieved, ground_truth):

    q = embed(query)
    r = embed(retrieved)
    gt = embed(ground_truth)

    # Query ↔ Retrieval
    sim1 = util.cos_sim(q, r).item()

    # Retrieval ↔ Ground truth
    sim2 = util.cos_sim(r, gt).item()

    return (sim1 + sim2) / 2


In [ ]:
def keyword_score(retrieved, keywords):
    if not keywords:
        return 0
    return sum(kw.strip() in retrieved for kw in keywords) / len(keywords)


In [ ]:
def hybrid_score(query, retrieved, keywords, ground_truth):

    sem = semantic_score(query, retrieved, ground_truth)
    kw = keyword_score(retrieved, keywords)

    # Normalize semantic to 0–1 range if needed
    # Cosine similarity is roughly -1 to +1, but E5 outputs are mostly 0.3–0.9
    sem_norm = max(0, min((sem + 1) / 2, 1))

    # Hybrid blend
    score = (sem_norm * 0.8) + (kw * 0.2)

    return round(score * 100, 2)   # return score as 0–100


In [ ]:
results = []

for item in tqdm(EVAL_SET):
    q = item["query"]
    kw = item["keywords"]
    gt = item["ground_truth"]

    retrieved = retrieve(q, k=3)

    docs = retrieved["documents"]
    top_text = docs[0][0] if isinstance(docs[0], list) else docs[0]

    score = hybrid_score(q, top_text, kw, gt)

    results.append({
        "query": q,
        "retrieved": top_text,
        "score": score
    })

100%|██████████| 2/2 [00:05<00:00,  2.53s/it]


In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv("retrieval_eval_results.csv", index=False)

df


query  \
0                    ما هي شروط التخرج؟   
1  متى يحرم الطالب من الاختبار النهائي؟   

                                           retrieved  score  
0  يتخرج الطالب بعد إنهاء متطلبات التخرج بنجاح حس...  93.48  
1  على الطالب المنتظم حضور المحاضرات والدروس العم...  84.16